# Data loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
feature=pd.read_csv('finalfeatures.csv')
amount=pd.read_csv('费用类相关特征2.csv')
amount=amount.set_index('个人编码')
amount=amount[['总审批金额']]
feature=feature.set_index('个人编码')
feature=feature.drop(columns=['总审批金额'])
feature=pd.concat([feature,amount],axis=1)
feature = feature.fillna(0)

In [3]:
# Bayes Network features
bayesdata=pd.read_csv('bayesdata2.csv')
bayesdata=bayesdata.set_index('个人编码')
feature=pd.concat([feature,bayesdata],axis=1)

# BN-ICXGB 

In [4]:
# cost-sensitive XGB
import xgboost as xgb
from scipy.special import expit

class CSBoost:
    def __init__(self, obj, lambda1=0, lambda2=0, learn_rate=0.01):
        self.obj = obj
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.learn_rate=learn_rate
        # alpha is l1, lambda is l2
        params = {'random_state': 42, 'tree_method': 'exact', 'verbosity': 0, 'reg_alpha': lambda1,
                  'reg_lambda': lambda2,'learning_rate': learn_rate}
        if obj == 'ce' or obj == 'weightedce':
            params['objective'] = 'binary:logistic'
        elif obj == 'aec':
            params['disable_default_eval_metric'] = True

        self.params = params

    def fit(self, x_train, y_train, x_val, y_val, cost_matrix_train=None, cost_matrix_val=None):
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        x_val = np.array(x_val)
        y_val = np.array(y_val)
        cost_matrix_train = np.array(cost_matrix_train)
        cost_matrix_val = np.array(cost_matrix_val)
        if self.obj == 'ce':
            dtrain = xgb.DMatrix(x_train, label=y_train)
            dval = xgb.DMatrix(x_val, label=y_val)

            xgboost = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=500, early_stopping_rounds=50,
                            evals=[(dval, 'eval')], verbose_eval=False)

        elif self.obj == 'weightedce':
            misclass_costs = np.zeros(len(y_train))
            misclass_costs[y_train == 0] = cost_matrix_train[:, 1, 0][y_train == 0]
            misclass_costs[y_train == 1] = cost_matrix_train[:, 0, 1][y_train == 1]

            misclass_costs_val = np.zeros(len(y_val))
            misclass_costs_val[y_val == 0] = cost_matrix_val[:, 1, 0][y_val == 0]
            misclass_costs_val[y_val == 1] = cost_matrix_val[:, 0, 1][y_val == 1]

            dtrain = xgb.DMatrix(x_train, label=y_train, weight=misclass_costs)
            dval = xgb.DMatrix(x_val, label=y_val, weight=misclass_costs_val)

            xgboost = xgb.train(params=self.params, dtrain=dtrain, num_boost_round=500, early_stopping_rounds=50,
                                evals=[(dval, 'eval')], verbose_eval=False)

        elif self.obj == 'aec':
            dtrain = xgb.DMatrix(x_train, label=y_train)
            dval = xgb.DMatrix(x_val, label=y_val)

            # Do constant computations here to avoid DMatrix error
            # diff_costs_train = fixed_cost - y_train * amounts_train

            train_constant = (y_train * (cost_matrix_train[:, 1, 1] - cost_matrix_train[:, 0, 1])
                              + (1 - y_train) * (cost_matrix_train[:, 1, 0] - cost_matrix_train[:, 0, 0]))

            def aec_train(raw_scores, y_true):
                scores = expit(raw_scores)

                # Average expected cost:
                # ec = np.multiply(np.multiply(y_true, (1 - scores)), amounts_train) + np.multiply(scores, fixed_cost)
                # ec = y_true * (
                #     scores * cost_matrix_train[:, 1, 1] + (1 - scores) * cost_matrix_val[:, 0, 1]) \
                #     + (1 - y_true) * (
                #     scores * cost_matrix_val[:, 1, 0] + (1 - scores) * cost_matrix_val[:, 0, 0])

                # Gradient
                # Use diff_costs_train instead of (fixed_cost - y_true*amounts_train)
                # grad = scores * (1 - scores) * diff_costs_train
                grad = scores * (1 - scores) * train_constant

                # Hessian
                hess = np.abs((1 - 2 * scores) * grad)
                # hess = scores * (1 - scores) * (1 - 2 * scores) * train_constant

                # Grad and hess cannot be too close to 0!
                # print(grad.mean())
                # print(hess.mean())

                return grad, hess

            def aec_val(raw_scores, y_true):
                scores = expit(raw_scores)

                # Return AEC (not grad/hess)
                # ec = (1 - scores) * y_val * amounts_val + scores * fixed_cost
                # ec = y_true * (
                #     scores * cost_matrix_val[:, 1, 1] + (1 - scores) * cost_matrix_val[:, 0, 1]) \
                #     + (1 - y_true) * (
                #     scores * cost_matrix_val[:, 1, 0] + (1 - scores) * cost_matrix_val[:, 0, 0])

                # Avoid computations with y_true (DMatrix)
                if y_true:
                    ec = scores * cost_matrix_val[:, 1, 1] + (1 - scores) * cost_matrix_val[:, 0, 1]
                else:
                    ec = scores * cost_matrix_val[:, 1, 0] + (1 - scores) * cost_matrix_val[:, 0, 0]

                aec = ec.mean()

                return 'AEC', aec

            xgboost = xgb.train(params=self.params, dtrain=dtrain, obj=aec_train, feval=aec_val, num_boost_round=500,
                                early_stopping_rounds=50, evals=[(dval, 'eval')], verbose_eval=False)

        # print('\tBest number of trees = %i' % xgboost.best_ntree_limit)

        return xgboost

    def tune(self, l1, lambda1_list, l2, lambda2_list, learn_rate, learn_ratelist, x_train, y_train, cost_matrix_train, x_val, y_val, cost_matrix_val):
        if l1:
            self.params['reg_lambda'] = 0
            losses_list = []
            for lambda1 in lambda1_list:
                xgboost = CSBoost(obj=self.obj, lambda1=lambda1)
                xgboost = xgboost.fit(x_train, y_train, x_val, y_val, cost_matrix_train, cost_matrix_val)
                scores = xgboost.inplace_predict(x_val)

                # Evaluate loss (without regularization term!)
                if self.obj == 'ce':
                    eps = 1e-9  # small value to avoid log(0)
                    ce = - (y_val * np.log(scores + eps) + (1 - y_val) * np.log(1 - scores + eps))
                    val_loss = ce.mean()
                elif self.obj == 'weightedce':
                    eps = 1e-9  # small value to avoid log(0)
                    ce = - (y_val * np.log(scores + eps) + (1 - y_val) * np.log(1 - scores + eps))

                    cost_misclass = np.zeros(len(y_val))
                    cost_misclass[y_val == 0] = cost_matrix_val[:, 1, 0][y_val == 0]
                    cost_misclass[y_val == 1] = cost_matrix_val[:, 0, 1][y_val == 1]

                    weighted_ce = cost_misclass * ce
                    val_loss = weighted_ce.mean()
                elif self.obj == 'aec':
                    def aec_val(raw_scores, y_true):
                        scores = expit(raw_scores)

                        # Return AEC (not grad/hess)
                        # ec = (1 - scores) * y_val * amounts_val + scores * fixed_cost
                        ec = y_true * (
                            scores * cost_matrix_val[:, 1, 1] + (1 - scores) * cost_matrix_val[:, 0, 1]) \
                            + (1 - y_true) * (
                            scores * cost_matrix_val[:, 1, 0] + (1 - scores) * cost_matrix_val[:, 0, 0])

                        aec = ec.mean()

                        return 'AEC', aec

                    aec = aec_val(scores, y_val)
                    val_loss = aec[1]
                print('\t\tLambda l1 = %.5f;\tLoss = %.5f' % (lambda1, val_loss))
                losses_list.append(val_loss)
            lambda1_opt = lambda1_list[np.argmin(losses_list)]
            print('\tOptimal lambda = %.5f' % lambda1_opt)
            self.params['reg_alpha'] = lambda1_opt
        elif l2:
            self.params['reg_alpha'] = 0
            losses_list = []
            for lambda2 in lambda2_list:
                xgboost = CSBoost(obj=self.obj, lambda2=lambda2)
                xgboost = xgboost.fit(x_train, y_train, x_val, y_val, cost_matrix_train, cost_matrix_val)
                scores = xgboost.inplace_predict(x_val)

                # Evaluate loss (without regularization term!)
                if self.obj == 'ce':
                    eps = 1e-9  # small value to avoid log(0)
                    ce = - (y_val * np.log(scores + eps) + (1 - y_val) * np.log(1 - scores + eps))
                    val_loss = ce.mean()
                elif self.obj == 'weightedce':
                    eps = 1e-9  # small value to avoid log(0)
                    ce = - (y_val * np.log(scores + eps) + (1 - y_val) * np.log(1 - scores + eps))

                    cost_misclass = np.zeros(len(y_val))
                    cost_misclass[y_val == 0] = cost_matrix_val[:, 1, 0][y_val == 0]
                    cost_misclass[y_val == 1] = cost_matrix_val[:, 0, 1][y_val == 1]

                    weighted_ce = cost_misclass * ce
                    val_loss = weighted_ce.mean()
                elif self.obj == 'aec':
                    def aec_val(raw_scores, y_true):
                        scores = expit(raw_scores)

                        # Return AEC (not grad/hess)
                        ec = y_true * (
                                scores * cost_matrix_val[:, 1, 1] + (1 - scores) * cost_matrix_val[:, 0, 1]) \
                             + (1 - y_true) * (
                                     scores * cost_matrix_val[:, 1, 0] + (1 - scores) * cost_matrix_val[:, 0, 0])

                        aec = ec.mean()

                        return 'AEC', aec

                    aec = aec_val(scores, y_val)
                    val_loss = aec[1]
                print('\t\tLambda l2 = %.5f;\tLoss = %.5f' % (lambda2, val_loss))
                losses_list.append(val_loss)
            lambda2_opt = lambda2_list[np.argmin(losses_list)]
            print('\tOptimal lambda = %.5f' % lambda2_opt)
            self.params['reg_alpha'] = lambda2_opt
        elif learn_rate:
            self.params['learning_rate'] = 0.01
            losses_list = []
            for learn_rate in learn_ratelist:
                xgboost = CSBoost(obj=self.obj, learn_rate=learn_rate)
                xgboost = xgboost.fit(x_train, y_train, x_val, y_val, cost_matrix_train, cost_matrix_val)
                scores = xgboost.inplace_predict(x_val)

                # Evaluate loss (without regularization term!)
                if self.obj == 'ce':
                    eps = 1e-9  # small value to avoid log(0)
                    ce = - (y_val * np.log(scores + eps) + (1 - y_val) * np.log(1 - scores + eps))
                    val_loss = ce.mean()
                elif self.obj == 'weightedce':
                    eps = 1e-9  # small value to avoid log(0)
                    ce = - (y_val * np.log(scores + eps) + (1 - y_val) * np.log(1 - scores + eps))

                    cost_misclass = np.zeros(len(y_val))
                    cost_misclass[y_val == 0] = cost_matrix_val[:, 1, 0][y_val == 0]
                    cost_misclass[y_val == 1] = cost_matrix_val[:, 0, 1][y_val == 1]

                    weighted_ce = cost_misclass * ce
                    val_loss = weighted_ce.mean()
                elif self.obj == 'aec':
                    def aec_val(raw_scores, y_true):
                        scores = expit(raw_scores)

                        # Return AEC (not grad/hess)
                        # ec = (1 - scores) * y_val * amounts_val + scores * fixed_cost
                        ec = y_true * (
                            scores * cost_matrix_val[:, 1, 1] + (1 - scores) * cost_matrix_val[:, 0, 1]) \
                            + (1 - y_true) * (
                            scores * cost_matrix_val[:, 1, 0] + (1 - scores) * cost_matrix_val[:, 0, 0])

                        aec = ec.mean()

                        return 'AEC', aec

                    aec = aec_val(scores, y_val)
                    val_loss = aec[1]
                print('\t\tlearning_rate = %.5f;\tLoss = %.5f' % (learn_rate, val_loss))
                losses_list.append(val_loss)
            learn_rate_opt = learn_ratelist[np.argmin(losses_list)]
            print('\tOptimal learn_rate = %.5f' % learn_rate_opt)
            self.params['learn_rate'] = learn_rate_opt
        else:
            self.lambda1 = 0
            self.lambda2 = 0
            self.learn_rate=0.15

In [5]:
# Grid search for hyperparameters

y_train=feature['欺诈状态']
x_train=feature.drop(columns=['欺诈状态'])

#Split the train and test dataset
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val=train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=333)

#Create cost matrix
amounts_train = x_train['总审批金额'].values
amounts_val = x_val['总审批金额'].values

cost_matrix_train = np.zeros((len(x_train), 2, 2))     # cost_matrix [[TN, FN], [FP, TP]]
cost_matrix_train[:, 0, 0] = 0.0
cost_matrix_train[:, 0, 1] = amounts_train
cost_matrix_train[:, 1, 0] = 20000
cost_matrix_train[:, 1, 1] = 15000

cost_matrix_val = np.zeros((len(x_val), 2, 2))     # cost_matrix [[TN, FN], [FP, TP]]
cost_matrix_val[:, 0, 0] = 0.0
cost_matrix_val[:, 0, 1] = amounts_val
cost_matrix_val[:, 1, 0] = 20000
cost_matrix_val[:, 1, 1] = 15000

#Train model
csboost = CSBoost(obj='aec')
csboost.tune(False, [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], False, [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], True,[0.01,0.02,0.05,0.1,0.15], x_train, y_train,cost_matrix_train, x_val, y_val, cost_matrix_val)
# lambda1 = 0.01
# lambda2 = 0.00001
# learn_rate=0.1
#csboost=csboost.fit(x_train, y_train, x_val, y_val, cost_matrix_train, cost_matrix_val)

D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.01000;	Loss = 4829.09912


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.02000;	Loss = 1300.65364


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.05000;	Loss = 1216.30604


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.10000;	Loss = 1053.50505


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.15000;	Loss = 1096.79378
	Optimal learn_rate = 0.10000


In [6]:
# Savings metric
def cost_with_algorithm(cost_matrix, labels, predictions):

    cost_tn = cost_matrix[:, 0, 0][np.logical_and(predictions == 0, labels == 0)].sum()
    cost_fn = cost_matrix[:, 0, 1][np.logical_and(predictions == 0, labels == 1)].sum()
    cost_fp = cost_matrix[:, 1, 0][np.logical_and(predictions == 1, labels == 0)].sum()
    cost_tp = cost_matrix[:, 1, 1][np.logical_and(predictions == 1, labels == 1)].sum()

    return sum((cost_tn, cost_fn, cost_fp, cost_tp))

def cost_without_algorithm(cost_matrix, labels):

    # Predict everything as the default class that leads to minimal cost
    # Also include cost of TP/TN!
    cost_neg = cost_matrix[:, 0, 0][labels == 0].sum() + cost_matrix[:, 0, 1][labels == 1].sum()
    cost_pos = cost_matrix[:, 1, 0][labels == 0].sum() + cost_matrix[:, 1, 1][labels == 1].sum()

    return min(cost_neg, cost_pos)

def savings(cost_matrix, labels, predictions):
    
    cost_without = cost_without_algorithm(cost_matrix, labels)
    cost_with = cost_with_algorithm(cost_matrix, labels, predictions)
    savings = 1 - cost_with / cost_without
    
    return savings

In [7]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import classification_report

# Prepare the data
train = feature
y = train['欺诈状态']
X = train.drop(columns=['欺诈状态'])

results_df = pd.DataFrame(columns=['Fold', 'Recall', 'F1-score', 'Precision', 'Savings'])

# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1000)

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    
    # Split the train and test dataset
    x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Creat cost matrix
    amounts_train = x_train['总审批金额'].values
    amounts_val = x_val['总审批金额'].values
    
    cost_matrix_train = np.zeros((len(x_train), 2, 2))
    cost_matrix_train[:, 0, 0] = 0
    cost_matrix_train[:, 0, 1] = amounts_train
    cost_matrix_train[:, 1, 0] = 20000
    cost_matrix_train[:, 1, 1] = 15000
    
    cost_matrix_val = np.zeros((len(x_val), 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Train model
    csboost = CSBoost(obj='aec')
    lambda1 = 0.1
    lambda2 = 0.001
    learn_rate=0.15
    csboost = csboost.fit(x_train, y_train, x_val, y_val, cost_matrix_train, cost_matrix_val)
    
    # Obtain the predictions
    pre_val = expit(csboost.inplace_predict(x_val))
    threshold_instance = (cost_matrix_val[:, 1, 0] - cost_matrix_val[:, 0, 0]) / (
        cost_matrix_val[:, 1, 0] - cost_matrix_val[:, 0, 0]
        + cost_matrix_val[:, 0, 1] - cost_matrix_val[:, 1, 1])
    pred = (pre_val > threshold_instance).astype(int)
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings(cost_matrix_val, y_val, pred), 4)
    }
    
    results_df = pd.concat([results_df, pd.DataFrame([fold_results])], ignore_index=True)

D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_29632\604130138.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([fold_results])], ignore_index=True)
D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest

In [354]:
# results_df.to_csv('results-bnicxgb/bnicxgb10.csv', index=False)

# Basic features

In [8]:
# Grid search for hyperparameters
from sklearn.preprocessing import MinMaxScaler

# Prepare the data
amount = pd.read_csv('费用类相关特征2.csv')
amount = amount[['个人编码', '检查费发生金额', '检查费申报金额']]
train = feature.copy()
train = train.reset_index()  
train = pd.merge(train, amount, 
                on='个人编码',  
                how='left')     

# Fill in missing values
train = train.fillna(0)

# Prepare the features and label for model
x_train = train[['药品费申报金额','药品费发生金额','检查费发生金额','检查费申报金额',
                 '治疗费发生金额','治疗费申报金额','基本医疗保险统筹基金支付金额',
                 '起付标准以上自负比例金额','总审批金额']]
y_train = train['欺诈状态']

# Normalization
scaler = MinMaxScaler(feature_range=(0, 1))
x_train['检查费发生金额'] = scaler.fit_transform(x_train[['检查费发生金额']])
x_train['检查费申报金额'] = scaler.fit_transform(x_train[['检查费申报金额']])

#Split the train and test dataset
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val=train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=333)

#Create cost matrix
amounts_train = x_train['总审批金额'].values
amounts_val = x_val['总审批金额'].values

cost_matrix_train = np.zeros((len(x_train), 2, 2))     # cost_matrix [[TN, FN], [FP, TP]]
cost_matrix_train[:, 0, 0] = 0.0
cost_matrix_train[:, 0, 1] = amounts_train
cost_matrix_train[:, 1, 0] = 20000
cost_matrix_train[:, 1, 1] = 15000

cost_matrix_val = np.zeros((len(x_val), 2, 2))     # cost_matrix [[TN, FN], [FP, TP]]
cost_matrix_val[:, 0, 0] = 0.0
cost_matrix_val[:, 0, 1] = amounts_val
cost_matrix_val[:, 1, 0] = 20000
cost_matrix_val[:, 1, 1] = 15000

#Train model
csboost = CSBoost(obj='aec')
csboost.tune(True, [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], False, [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], False,[0.01,0.02,0.05,0.1,0.15], x_train, y_train,cost_matrix_train, x_val, y_val, cost_matrix_val)
# lambda1 = 0.1
# lambda2 = 0.1
# learn_rate=0.15

C:\Users\Administrator\AppData\Local\Temp\ipykernel_29632\638139466.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['检查费发生金额'] = scaler.fit_transform(x_train[['检查费发生金额']])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_29632\638139466.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['检查费申报金额'] = scaler.fit_transform(x_train[['检查费申报金额']])
D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different be

		Lambda l1 = 0.00000;	Loss = 4889.13103
		Lambda l1 = 0.00001;	Loss = 4889.13103


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(
D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		Lambda l1 = 0.00010;	Loss = 4889.13104
		Lambda l1 = 0.00100;	Loss = 4889.13109


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(
D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		Lambda l1 = 0.01000;	Loss = 4888.88931
		Lambda l1 = 0.10000;	Loss = 4889.24173
	Optimal lambda = 0.01000


In [9]:
# Prepare the features and label for model
X=train[['药品费申报金额','药品费发生金额','检查费发生金额','检查费申报金额','治疗费发生金额','治疗费申报金额','基本医疗保险统筹基金支付金额','起付标准以上自负比例金额','总审批金额']]
y=train['欺诈状态']
results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])

# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1000)

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    
    # Split the train and test dataset
    x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Creat the cost matrix
    amounts_train = x_train['总审批金额'].values
    amounts_val = x_val['总审批金额'].values
    
    cost_matrix_train = np.zeros((len(x_train), 2, 2))
    cost_matrix_train[:, 0, 0] = 0
    cost_matrix_train[:, 0, 1] = amounts_train
    cost_matrix_train[:, 1, 0] = 20000
    cost_matrix_train[:, 1, 1] = 15000
    
    cost_matrix_val = np.zeros((len(x_val), 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Train model
    csboost = CSBoost(obj='aec')
    lambda1 = 0.1
    lambda2 = 0.1
    learn_rate=0.15
    csboost = csboost.fit(x_train, y_train, x_val, y_val, cost_matrix_train, cost_matrix_val)
    
    # Obain the predictions
    pre_val = expit(csboost.inplace_predict(x_val))
    threshold_instance = (cost_matrix_val[:, 1, 0] - cost_matrix_val[:, 0, 0]) / (
        cost_matrix_val[:, 1, 0] - cost_matrix_val[:, 0, 0]
        + cost_matrix_val[:, 0, 1] - cost_matrix_val[:, 1, 1])
    pred = (pre_val > threshold_instance).astype(int)
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings(cost_matrix_val, y_val, pred),4)
    }
    
    results_df = pd.concat([results_df, pd.DataFrame([fold_results])], ignore_index=True)

D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_29632\876652144.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([fold_results])], ignore_index=True)
D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest

In [42]:
results_df.to_csv('results-basicfeatures/basicfeatures10.csv', index=False)

# BN

In [10]:
from pgmpy.estimators import BicScore, ExhaustiveSearch, HillClimbSearch
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator

train=feature
bayes=train[['主要发生金额','就诊总费用','总审批金额','主要申报金额','滑窗60-3','基本医疗保险统筹基金支付金额极值60','起付标准以上自负比例金额极差60','欺诈状态']]

# Discretization of continuous data
from sklearn.preprocessing import KBinsDiscretizer
temp_X = bayes[['主要发生金额','就诊总费用','总审批金额','主要申报金额','滑窗60-3','基本医疗保险统筹基金支付金额极值60','起付标准以上自负比例金额极差60']].values
Kbins = KBinsDiscretizer(n_bins = 2, encode = 'ordinal', strategy = 'kmeans')
X_Kbins = Kbins.fit_transform(temp_X)
temp_X = bayes[['主要发生金额','就诊总费用','总审批金额','主要申报金额','滑窗60-3','基本医疗保险统筹基金支付金额极值60','起付标准以上自负比例金额极差60']] = np.int8(X_Kbins)

results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])

# 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(bayes)):
    
    # Split the train and test data
    model_train = bayes.iloc[train_index]
    test = bayes.iloc[val_index]
    model_test = test.drop(['欺诈状态'], axis=1)
    
    # Construct and train model
    hc = HillClimbSearch(model_train)
    best_model = hc.estimate()
    best_model = BayesianModel(best_model.edges())
    best_model.fit(data=model_train, estimator=BayesianEstimator)
    
    # Obtain the predictions
    best_model_pre = best_model.predict(model_test, n_jobs=5)
    scores = best_model_pre
    y_pred = (scores >= 0.5)*1
    
    # Calculate the Savings metric
    fee = pd.DataFrame()
    fee['数额'] = train['总审批金额'].iloc[val_index]
    fee.index = test.index  # 确保索引匹配
    fee = pd.concat([test, fee], axis=1)
    
    # Create cost marix
    cost_matrix_val = np.zeros((len(x_val), 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    y_val = y_val.values.flatten()  
    y_val = pd.Series(y_val, name='label')
    y_val = pd.DataFrame(y_val).reset_index(drop=True)
    best_model_pre = pd.DataFrame(best_model_pre, columns=['欺诈状态']).reset_index(drop=True)
    yz = pd.concat([y_val, best_model_pre], axis=1)
    
    saving = savings(cost_matrix_val, yz['label'], yz['欺诈状态'])
    
    # Obtain the performance metricx
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(yz['label'], yz['欺诈状态']), 4),
        'Recall': round(metrics.recall_score(yz['label'], yz['欺诈状态']), 4),
        'F1-score': round(metrics.f1_score(yz['label'], yz['欺诈状态']), 4),
        'Precision': round(metrics.precision_score(yz['label'], yz['欺诈状态']), 4),
        'Savings': round(saving, 4)
    }
    
    results_df.loc[len(results_df)] = fold_results

#results_df.to_csv('results-bn/bn10.csv', index=False)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_29632\1654312376.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_X = bayes[['主要发生金额','就诊总费用','总审批金额','主要申报金额','滑窗60-3','基本医疗保险统筹基金支付金额极值60','起付标准以上自负比例金额极差60']] = np.int8(X_Kbins)


  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

# ICXGB

In [11]:
# Grid search for hyperparameters

# Prepare the train and test dataset
train = feature.iloc[:, :150]
y_train = train['欺诈状态']
x_train = train.drop(columns=['欺诈状态'])
x_train, x_val, y_train, y_val=train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=333)
amounts_train = x_train['总审批金额'].values
amounts_val = x_val['总审批金额'].values

# Create cost matrix
cost_matrix_train = np.zeros((len(x_train), 2, 2))     # cost_matrix [[TN, FN], [FP, TP]]
cost_matrix_train[:, 0, 0] = 0
cost_matrix_train[:, 0, 1] = amounts_train
cost_matrix_train[:, 1, 0] = 20000
cost_matrix_train[:, 1, 1] = 15000
cost_matrix_val = np.zeros((len(x_val), 2, 2))     # cost_matrix [[TN, FN], [FP, TP]]
cost_matrix_val[:, 0, 0] = 0
cost_matrix_val[:, 0, 1] = amounts_val
cost_matrix_val[:, 1, 0] = 20000
cost_matrix_val[:, 1, 1] = 15000

# Train model          
csboost = CSBoost(obj='aec')
csboost.tune(False, [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], False, [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], True, [0.01,0.02,0.05,0.1,0.15], x_train, y_train,cost_matrix_train, x_val, y_val, cost_matrix_val)
# lambda1 =0.001
# lambda2 =0.0001
# learn_rate=0.1

D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.01000;	Loss = 4840.25772


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.02000;	Loss = 1248.38718


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.05000;	Loss = 1281.29254


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.10000;	Loss = 1055.97155


D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


		learning_rate = 0.15000;	Loss = 1103.56474
	Optimal learn_rate = 0.10000


In [13]:
# Prepare the data
X=feature.iloc[:, :150]
y=feature['欺诈状态']

results_df = pd.DataFrame(columns=['Fold', 'Recall', 'F1-score', 'Precision', 'Savings'])

# 5-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    
    # Split the train and test dataset
    x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Construct the cost matrix
    amounts_train = x_train['总审批金额'].values
    amounts_val = x_val['总审批金额'].values
    
    cost_matrix_train = np.zeros((len(x_train), 2, 2))
    cost_matrix_train[:, 0, 0] = 0
    cost_matrix_train[:, 0, 1] = amounts_train
    cost_matrix_train[:, 1, 0] = 20000
    cost_matrix_train[:, 1, 1] = 15000
    
    cost_matrix_val = np.zeros((len(x_val), 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Train model
    csboost = CSBoost(obj='aec')
    lambda1 =0.001
    lambda2 =0.0001
    learn_rate=0.15
    csboost = csboost.fit(x_train, y_train, x_val, y_val, cost_matrix_train, cost_matrix_val)
    
    # Obtain the predictions
    pre_val = expit(csboost.inplace_predict(x_val))
    threshold_instance = (cost_matrix_val[:, 1, 0] - cost_matrix_val[:, 0, 0]) / (
        cost_matrix_val[:, 1, 0] - cost_matrix_val[:, 0, 0]
        + cost_matrix_val[:, 0, 1] - cost_matrix_val[:, 1, 1])
    pred = (pre_val > threshold_instance).astype(int)
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings(cost_matrix_val, y_val, pred), 4)
    }
    
    results_df = pd.concat([results_df, pd.DataFrame([fold_results])], ignore_index=True)

D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_29632\2087295910.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([fold_results])], ignore_index=True)
D:\anaconda\Lib\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/lates

In [14]:
#results_df.to_csv('results-icxgb/icxgb10.csv', index=False)

# LR 

In [15]:
from sklearn.linear_model import LinearRegression

results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])
# Prepare the data
train = feature
y = train['欺诈状态']
X = train.drop(columns=['欺诈状态'])

# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=333)

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    
    # Split the train and test dataset
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Train model
    reg = LinearRegression()
    reg.fit(X_train, y_train)
    
    # Obtain the predictions
    pred = reg.predict(X_val)
    threshold_class_imbalance = 0.5
    pred = (pred > threshold_class_imbalance).astype(int)
    
    # Create the cost matrix
    fee = pd.DataFrame()
    fee['数额'] = train['总审批金额'].iloc[val_idx]
    amounts_val = fee['数额'].values
    
    cost_matrix_val = np.zeros((len(X_val), 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings(cost_matrix_val, y_val, pred), 4)
    }
    
    results_df.loc[len(results_df)] = fold_results

#results_df.to_csv('results-lr/lr1.csv', index=False, float_format='%.4f')

# SVM

In [16]:
from sklearn.svm import SVC
results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])

# Prepare data
train = feature
y = train['欺诈状态']
X = train.drop(columns=['欺诈状态'])

# 5-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X)):    
    
    # Split the train and test dataset
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]    
   
    # Train model
    clf = SVC(kernel='rbf')
    clf.fit(X_train, y_train)    
    # Obtain the predictions
    pred = clf.predict(X_val)    
   
    # Create the cost matrix
    fee = pd.DataFrame()
    fee['数额'] = train['总审批金额'].iloc[val_idx]
    amounts_val = fee['数额'].values 
    cost_matrix_val = np.zeros((len(X_val), 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings(cost_matrix_val, y_val, pred), 4)
    }
    results_df.loc[len(results_df)] = fold_results    
# Save results
#results_df.to_csv('results-svm/svm5.csv', index=False, float_format='%.4f')

# XGB

In [17]:
# Grid search for hyperparameters
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
train=feature.iloc[:, :150]
y_train=train['欺诈状态']
x_train=train.drop(columns=['欺诈状态'])
x_train,x_val,y_train,y_val=train_test_split(x_train, y_train,test_size=0.2,shuffle=True,random_state=333)
cv_params = {'reg_lambda': [0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]}
other_params = {'min_child_weight': 1, 'seed': 0, 'n_estimators':500,'learning_rate':0.1,'max_depth': 20,'reg_alpha':0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0}
tree_model = XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=tree_model,param_grid=cv_params,scoring='r2',cv=5,verbose=1, n_jobs=4)
grid_result = optimized_GBM.fit(x_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Best: 0.200783 using {'reg_lambda': 0.001}


In [18]:
# Model hyper-parameters
params = {
    'min_child_weight': 1, 
    'seed': 0, 
    'n_estimators': 500,
    'max_depth': 20,
    'learning_rate': 0.1,
    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
    'gamma': 0, 
    'reg_alpha': 0, 
    'reg_lambda': 1
}
results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])

# Prepare data
train=feature.iloc[:, :150]
y_train=train['欺诈状态']
x_train=train.drop(columns=['欺诈状态'])

# 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    
    # Split the train and test dataset
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # Train model
    tree_model = XGBClassifier(**params)
    tree_model.fit(X_train, y_train)
    
    # Obtain the predictions
    pred = tree_model.predict(X_val)
    
    # Create the cost matrix
    fee = pd.DataFrame()
    fee['数额'] = train['总审批金额'].iloc[val_idx]
    amounts_val = fee['数额'].values
    cost_matrix_val = np.zeros((len(X_val), 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000   
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings(cost_matrix_val, y_val, pred), 4)
    }
    results_df.loc[len(results_df)] = fold_results    
# Save results
#results_df.to_csv('results-xgb/xgb10.csv', index=False, float_format='%.4f')

# DNN

In [19]:
# Step 1: Import necessary libraries

from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from sklearn.model_selection import train_test_split


# Step 2: Prepare your data
train=feature.iloc[:, :150]
y_train=train['欺诈状态']
x_train=train.drop(columns=['欺诈状态'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=333)

# Step 3: Define a model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    
    # Hyperparameter for the number of neurons in the first hidden layer
    hp_units = hp.Int('units', min_value=32, max_value=64, step=32)
    model.add(Dense(units=hp_units, activation='relu', input_dim=x_train.shape[1]))
    
    # Hyperparameter for dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.2, step=0.1)
    model.add(Dropout(hp_dropout))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Hyperparameter for learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Step 4: Initialize Keras Tuner

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='intro_to_kt',
    overwrite=True
)

# Step 5: Run the hyperparameter search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Step 6: Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first densely-connected layer is {best_hps.get('units')}.
The optimal dropout rate is {best_hps.get('dropout')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.9512500166893005

Best val_accuracy So Far: 0.9514999985694885
Total elapsed time: 00h 00m 17s

The optimal number of units in the first densely-connected layer is 64.
The optimal dropout rate is 0.2.
The optimal learning rate for the optimizer is 0.001.



In [20]:
results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])
y = train['欺诈状态']
X = train.drop(columns=['欺诈状态'])

# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=1000)
for fold, (train_index, val_index) in enumerate(kf.split(X)):    
    
    # Split the train and test dataset
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # Train model
    model = tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), verbose=1)
    
    # Obtain the predictions
    pred = (model.predict(X_val) > 0.05).astype("int32").flatten()  # 确保pred是一维数组
    y_val = y_val.values  # 转换为numpy数组
    
    # Create cost matrix
    fee = pd.DataFrame()
    fee['数额'] = train['总审批金额'].iloc[val_index]
    amounts_val = fee['数额'].values    
    n_samples = len(amounts_val)
    cost_matrix_val = np.zeros((n_samples, 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Savings metric
    def cost_with_algorithm1(cost_matrix, labels, predictions):
        cost = 0
        for i in range(len(labels)):
            cost += cost_matrix[i, predictions[i], labels[i]]
        return cost

    def cost_without_algorithm1(cost_matrix, labels):
        cost_all_negative = sum(cost_matrix[i, 0, labels[i]] for i in range(len(labels)))
        cost_all_positive = sum(cost_matrix[i, 1, labels[i]] for i in range(len(labels)))
        return min(cost_all_negative, cost_all_positive)

    def savings1(cost_matrix, labels, predictions):
        cost_without = cost_without_algorithm1(cost_matrix, labels)
        cost_with = cost_with_algorithm1(cost_matrix, labels, predictions)
        savings = 1 - cost_with / cost_without
        return savings
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings1(cost_matrix_val, y_val, pred), 4)
    }
    
    results_df.loc[len(results_df)] = fold_results
    
# Save results
#results_df.to_csv('results-dnn/dnn10.csv', index=False, float_format='%.4f')

Epoch 1/10


D:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - accuracy: 0.8524 - loss: 130.4891 - val_accuracy: 0.9542 - val_loss: 23.4734
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 0.8475 - loss: 21.8694 - val_accuracy: 0.9542 - val_loss: 0.4288
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.9104 - loss: 0.5390 - val_accuracy: 0.9542 - val_loss: 0.3141
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - accuracy: 0.9207 - loss: 0.4803 - val_accuracy: 0.9542 - val_loss: 0.2626
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 651us/step - accuracy: 0.9367 - loss: 0.3895 - val_accuracy: 0.9542 - val_loss: 0.2767
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - accuracy: 0.9441 - loss: 0.3531 - val_accuracy: 0.9542 - val_loss: 0.2350
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 652us/step - accuracy: 0.9454 - loss: 0.2814 - val_accuracy: 0.9542 - val_loss: 0.2226
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.9496 - loss: 0.2375 - val_acc

D:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 811us/step - accuracy: 0.8356 - loss: 181.7928 - val_accuracy: 0.9465 - val_loss: 42.5467
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 678us/step - accuracy: 0.8627 - loss: 43.3309 - val_accuracy: 0.9465 - val_loss: 8.7074
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 653us/step - accuracy: 0.8651 - loss: 3.5272 - val_accuracy: 0.9465 - val_loss: 0.3304
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.9247 - loss: 0.4782 - val_accuracy: 0.9465 - val_loss: 0.3111
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - accuracy: 0.9310 - loss: 0.4229 - val_accuracy: 0.9465 - val_loss: 0.2988
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - accuracy: 0.9403 - loss: 0.3874 - val_accuracy: 0.9465 - val_loss: 0.2898
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 648us/step - accuracy: 0.9457 - loss: 0.3341 - val_accuracy: 0.9465 - val_loss: 0.3338
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.9477 - loss: 0.2863 - val_acc

...

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.8594 - loss: 132.2688 - val_accuracy: 0.9525 - val_loss: 21.9329
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step - accuracy: 0.8490 - loss: 26.4343 - val_accuracy: 0.9525 - val_loss: 1.8919
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.8876 - loss: 1.3165 - val_accuracy: 0.9525 - val_loss: 0.3938
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step - accuracy: 0.9269 - loss: 0.4434 - val_accuracy: 0.9525 - val_loss: 0.2987
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.9289 - loss: 0.4288 - val_accuracy: 0.9525 - val_loss: 0.5772
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - accuracy: 0.9406 - loss: 0.3615 - val_accuracy: 0.9525 - val_loss: 0.2575
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - accuracy: 0.9402 - loss: 0.3223 - val_accuracy: 0.9525 - val_loss: 0.2443
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 656us/step - accuracy: 0.9500 - loss: 0.2697 - val_acc

D:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 789us/step - accuracy: 0.6507 - loss: 773.2514 - val_accuracy: 0.9457 - val_loss: 77.7722
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.8566 - loss: 96.4103 - val_accuracy: 0.9457 - val_loss: 29.4903
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.8508 - loss: 33.0281 - val_accuracy: 0.9457 - val_loss: 3.4975
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - accuracy: 0.8807 - loss: 2.6137 - val_accuracy: 0.9457 - val_loss: 0.6384
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - accuracy: 0.9270 - loss: 0.4459 - val_accuracy: 0.9457 - val_loss: 0.3909
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.9253 - loss: 0.4648 - val_accuracy: 0.9457 - val_loss: 0.4085
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - accuracy: 0.9371 - loss: 0.3980 - val_accuracy: 0.9457 - val_loss: 0.2922
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - accuracy: 0.9309 - loss: 0.3907 - val_a

D:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 819us/step - accuracy: 0.8682 - loss: 103.0213 - val_accuracy: 0.9510 - val_loss: 21.9135
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - accuracy: 0.8574 - loss: 20.5963 - val_accuracy: 0.9510 - val_loss: 0.9471
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 665us/step - accuracy: 0.8936 - loss: 0.8486 - val_accuracy: 0.9510 - val_loss: 0.3356
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - accuracy: 0.9326 - loss: 0.4095 - val_accuracy: 0.9510 - val_loss: 0.4094
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - accuracy: 0.9388 - loss: 0.3775 - val_accuracy: 0.9510 - val_loss: 0.2915
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.9401 - loss: 0.3578 - val_accuracy: 0.9510 - val_loss: 0.2576
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 0.9436 - loss: 0.3380 - val_accuracy: 0.9510 - val_loss: 0.2367
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - accuracy: 0.9471 - loss: 0.2680 - val_acc

# LSTM

In [21]:
from tensorflow.keras.layers import LSTM
# Prepare data
train = feature.iloc[:, :150]
y_train = train['欺诈状态']
x_train = train.drop(columns=['欺诈状态'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=333)

# Reshape the data for LSTM (samples, time steps, features)
x_train = np.reshape(x_train.values, (x_train.shape[0], 1, x_train.shape[1]))
x_val = np.reshape(x_val.values, (x_val.shape[0], 1, x_val.shape[1]))

# Define a model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    
    # First LSTM layer
    hp_units = hp.Int('units', min_value=32, max_value=64, step=32)
    model.add(LSTM(units=hp_units, 
                  return_sequences=True,
                  input_shape=(1, x_train.shape[2])))
    
    # Dropout after first LSTM
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.2, step=0.1)
    model.add(Dropout(hp_dropout))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Hyperparameter for learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    return model

# Initialize Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='lstm_kt',
    overwrite=True
)

# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first LSTM layer is {best_hps.get('units')}.
The optimal dropout rate is {best_hps.get('dropout')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")


Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.9512500166893005

Best val_accuracy So Far: 0.9512500166893005
Total elapsed time: 00h 00m 18s

The optimal number of units in the first LSTM layer is 64.
The optimal dropout rate is 0.1.
The optimal learning rate for the optimizer is 0.0001.



In [22]:
results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])
y = train['欺诈状态']
X = train.drop(columns=['欺诈状态'])

# 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    
    # Split the train and test dataset
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Reshape
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))

    # Train model
    model = tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, y_train, 
                       epochs=10, 
                       validation_data=(X_val, y_val), 
                       verbose=1)
    
    # Obtain the predictions
    pred = (model.predict(X_val) > 0.05).astype("int32").flatten()  # 确保pred是一维数组
    y_val = y_val.values  # 转换为numpy数组
    
    # Create the cost matrix
    fee = pd.DataFrame()
    fee['数额'] = train['总审批金额'].iloc[val_index]
    amounts_val = fee['数额'].values
    
    n_samples = len(amounts_val)
    cost_matrix_val = np.zeros((n_samples, 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings1(cost_matrix_val, y_val, pred), 4)
    }
    results_df.loc[len(results_df)] = fold_results
    
# Save results
#results_df.to_csv('results-lstm/lstm10.csv', index=False, float_format='%.4f')

Epoch 1/10


D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9247 - loss: 0.4074 - val_accuracy: 0.9542 - val_loss: 0.2159
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.9505 - loss: 0.2203 - val_accuracy: 0.9542 - val_loss: 0.1861
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - accuracy: 0.9526 - loss: 0.1927 - val_accuracy: 0.9542 - val_loss: 0.1859
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.9502 - loss: 0.1996 - val_accuracy: 0.9542 - val_loss: 0.1859
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - accuracy: 0.9454 - loss: 0.2145 - val_accuracy: 0.9542 - val_loss: 0.1859
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - accuracy: 0.9481 - loss: 0.2057 - val_accuracy: 0.9542 - val_loss: 0.1859
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.9496 - loss: 0.2013 - val_accuracy: 0.9542 - val_loss: 0.1857
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.9479 - loss: 0.2072 - val_accuracy:

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9466 - loss: 0.4360 - val_accuracy: 0.9465 - val_loss: 0.3087
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.9529 - loss: 0.2831 - val_accuracy: 0.9465 - val_loss: 0.2223
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.9505 - loss: 0.2115 - val_accuracy: 0.9465 - val_loss: 0.2111
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step - accuracy: 0.9517 - loss: 0.1990 - val_accuracy: 0.9465 - val_loss: 0.2089
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.9517 - loss: 0.1962 - val_accuracy: 0.9465 - val_loss: 0.2097
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.9494 - loss: 0.2030 - val_accuracy: 0.9465 - val_loss: 0.2103
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.9495 - loss: 0.2028 - val_accuracy: 0.9465 - val_loss: 0.2105
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step - accuracy: 0.9518 - loss: 0.1959 - val_accuracy:

...

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9291 - loss: 0.4729 - val_accuracy: 0.9525 - val_loss: 0.2336
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step - accuracy: 0.9479 - loss: 0.2380 - val_accuracy: 0.9525 - val_loss: 0.2005
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.9515 - loss: 0.2037 - val_accuracy: 0.9525 - val_loss: 0.1915
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - accuracy: 0.9513 - loss: 0.1961 - val_accuracy: 0.9525 - val_loss: 0.1914
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - accuracy: 0.9494 - loss: 0.2024 - val_accuracy: 0.9525 - val_loss: 0.1912
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - accuracy: 0.9515 - loss: 0.1953 - val_accuracy: 0.9525 - val_loss: 0.1912
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.9506 - loss: 0.1973 - val_accuracy: 0.9525 - val_loss: 0.1912
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - accuracy: 0.9490 - loss: 0.2029 - val_accuracy:

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9512 - loss: 0.3187 - val_accuracy: 0.9457 - val_loss: 0.2176
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 972us/step - accuracy: 0.9495 - loss: 0.2075 - val_accuracy: 0.9457 - val_loss: 0.2113
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.9505 - loss: 0.2007 - val_accuracy: 0.9457 - val_loss: 0.2109
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 936us/step - accuracy: 0.9555 - loss: 0.1845 - val_accuracy: 0.9457 - val_loss: 0.2137
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.9523 - loss: 0.1943 - val_accuracy: 0.9457 - val_loss: 0.2149
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - accuracy: 0.9499 - loss: 0.2022 - val_accuracy: 0.9457 - val_loss: 0.2134
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - accuracy: 0.9522 - loss: 0.1988 - val_accuracy: 0.9457 - val_loss: 0.2188
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.9510 - loss: 0.2009 - val_accuracy:

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9501 - loss: 0.2468 - val_accuracy: 0.9510 - val_loss: 0.1966
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.9501 - loss: 0.2008 - val_accuracy: 0.9510 - val_loss: 0.1984
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - accuracy: 0.9510 - loss: 0.1988 - val_accuracy: 0.9510 - val_loss: 0.1964
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 938us/step - accuracy: 0.9475 - loss: 0.2088 - val_accuracy: 0.9510 - val_loss: 0.1960
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - accuracy: 0.9470 - loss: 0.2093 - val_accuracy: 0.9510 - val_loss: 0.1960
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - accuracy: 0.9513 - loss: 0.1968 - val_accuracy: 0.9510 - val_loss: 0.1956
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.9531 - loss: 0.1901 - val_accuracy: 0.9510 - val_loss: 0.1959
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.9491 - loss: 0.2052 - val_accuracy:

# GRU

In [23]:
from tensorflow.keras.layers import GRU

# Prepare data
train = feature.iloc[:, :150]
y_train = train['欺诈状态']
x_train = train.drop(columns=['欺诈状态'])

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=333)

# Reshape the data for GRU (samples, time steps, features)
x_train = np.reshape(x_train.values, (x_train.shape[0], 1, x_train.shape[1]))
x_val = np.reshape(x_val.values, (x_val.shape[0], 1, x_val.shape[1]))

# Define a model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    
    # First GRU layer
    hp_units = hp.Int('units', min_value=32, max_value=64, step=32)
    model.add(GRU(units=hp_units, 
                 return_sequences=True,
                 input_shape=(1, x_train.shape[2])))
    
    # Dropout after first GRU
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.3, step=0.1)
    model.add(Dropout(hp_dropout))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Hyperparameter for learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    return model

# Initialize Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=1,
    directory='my_dir',
    project_name='gru_kt',
    overwrite=True
)

# Run the hyperparameter search
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))
 
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first GRU layer is {best_hps.get('units')}.
The optimal dropout rate is {best_hps.get('dropout')}.
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.9512500166893005

Best val_accuracy So Far: 0.9512500166893005
Total elapsed time: 00h 00m 18s

The optimal number of units in the first GRU layer is 32.
The optimal dropout rate is 0.0.
The optimal learning rate for the optimizer is 0.01.



In [24]:
results_df = pd.DataFrame(columns=['Fold', 'ACC', 'Recall', 'F1-score', 'Precision', 'Savings'])
y = train['欺诈状态']
X = train.drop(columns=['欺诈状态'])

# 5-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    
    # Split the train and test dataset
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    
    # Reshape
    X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
    X_val = np.reshape(X_val.values, (X_val.shape[0], 1, X_val.shape[1]))
    
    # Train model
    model = tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, y_train, 
                       epochs=10, 
                       validation_data=(X_val, y_val), 
                       verbose=1)
    
    # Obtain the predictions
    pred = (model.predict(X_val) > 0.05).astype("int32").flatten()  
    y_val = y_val.values  
    
    # Create the cost matrix
    fee = pd.DataFrame()
    fee['数额'] = train['总审批金额'].iloc[val_index]
    amounts_val = fee['数额'].values
    
    n_samples = len(amounts_val)
    cost_matrix_val = np.zeros((n_samples, 2, 2))
    cost_matrix_val[:, 0, 0] = 0
    cost_matrix_val[:, 0, 1] = amounts_val
    cost_matrix_val[:, 1, 0] = 20000
    cost_matrix_val[:, 1, 1] = 15000
    
    # Obtain the performance metrics
    fold_results = {
        'Fold': fold,
        'ACC': round(metrics.accuracy_score(y_val, pred), 4),
        'Recall': round(metrics.recall_score(y_val, pred), 4),
        'F1-score': round(metrics.f1_score(y_val, pred), 4),
        'Precision': round(metrics.precision_score(y_val, pred), 4),
        'Savings': round(savings1(cost_matrix_val, y_val, pred), 4)
    }
    results_df.loc[len(results_df)] = fold_results
    
# Save results
# results_df.to_csv('results-gru/gru10.csv', index=False, float_format='%.4f')

Epoch 1/10


D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9343 - loss: 0.2318 - val_accuracy: 0.9542 - val_loss: 0.1885
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 908us/step - accuracy: 0.9486 - loss: 0.2046 - val_accuracy: 0.9542 - val_loss: 0.1870
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.9481 - loss: 0.2057 - val_accuracy: 0.9542 - val_loss: 0.1874
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - accuracy: 0.9518 - loss: 0.1952 - val_accuracy: 0.9542 - val_loss: 0.1879
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - accuracy: 0.9487 - loss: 0.2058 - val_accuracy: 0.9542 - val_loss: 0.1858
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.9487 - loss: 0.2042 - val_accuracy: 0.9542 - val_loss: 0.1944
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.9448 - loss: 0.2151 - val_accuracy: 0.9542 - val_loss: 0.1867
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.9483 - loss: 0.2072 - val_accuracy:

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9381 - loss: 0.2282 - val_accuracy: 0.9465 - val_loss: 0.2101
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - accuracy: 0.9507 - loss: 0.1976 - val_accuracy: 0.9465 - val_loss: 0.2087
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.9515 - loss: 0.1965 - val_accuracy: 0.9465 - val_loss: 0.2087
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.9495 - loss: 0.2018 - val_accuracy: 0.9465 - val_loss: 0.2110
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.9530 - loss: 0.1917 - val_accuracy: 0.9465 - val_loss: 0.2091
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.9522 - loss: 0.1947 - val_accuracy: 0.9465 - val_loss: 0.2099
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - accuracy: 0.9482 - loss: 0.2076 - val_accuracy: 0.9465 - val_loss: 0.2230
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step - accuracy: 0.9507 - loss: 0.1986 - val_accuracy:

...

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9012 - loss: 0.2655 - val_accuracy: 0.9457 - val_loss: 0.2160
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.9551 - loss: 0.1838 - val_accuracy: 0.9457 - val_loss: 0.2109
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.9487 - loss: 0.2045 - val_accuracy: 0.9457 - val_loss: 0.2186
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - accuracy: 0.9548 - loss: 0.1870 - val_accuracy: 0.9457 - val_loss: 0.2182
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - accuracy: 0.9513 - loss: 0.1976 - val_accuracy: 0.9457 - val_loss: 0.2139
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - accuracy: 0.9488 - loss: 0.2046 - val_accuracy: 0.9457 - val_loss: 0.2123
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.9513 - loss: 0.1990 - val_accuracy: 0.9457 - val_loss: 0.2109
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.9523 - loss: 0.1935 - val_accuracy:

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9273 - loss: 0.2338 - val_accuracy: 0.9510 - val_loss: 0.1957
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - accuracy: 0.9494 - loss: 0.2043 - val_accuracy: 0.9510 - val_loss: 0.1986
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.9475 - loss: 0.2083 - val_accuracy: 0.9510 - val_loss: 0.1959
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.9501 - loss: 0.2002 - val_accuracy: 0.9510 - val_loss: 0.1974
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - accuracy: 0.9505 - loss: 0.1984 - val_accuracy: 0.9510 - val_loss: 0.1968
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - accuracy: 0.9510 - loss: 0.1976 - val_accuracy: 0.9510 - val_loss: 0.1970
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step - accuracy: 0.9515 - loss: 0.1955 - val_accuracy: 0.9510 - val_loss: 0.2026
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step - accuracy: 0.9471 - loss: 0.2116 - val_accuracy:

D:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
